In [318]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/UTTARABANK.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP  TRADE  VALUE (mn)     VOLUME
0  439 2018-05-08   UTTARABANK  24.3  ...  24.8  171.0       4.281   175638.0
1  438 2018-06-08   UTTARABANK  24.1  ...  24.5  141.0       5.471   228028.0
2  437 2018-07-08   UTTARABANK  24.6  ...  24.0  140.0       5.201   213059.0
3  436 2018-08-08   UTTARABANK  25.3  ...  24.5  360.0      11.565   462117.0
4  435 2018-09-08   UTTARABANK  26.0  ...  25.2  700.0      32.533  1272906.0

[5 rows x 12 columns]

In [319]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP  TRADE  VALUE (mn)     VOLUME
36  403 2018-01-10   UTTARABANK  24.9  ...  25.0  248.0       6.841   271956.0
59  380 2018-01-11   UTTARABANK  29.8  ...  29.5  751.0      44.419  1522151.0
37  402 2018-02-10   UTTARABANK  25.4  ...  25.1  150.0       4.143   164334.0
79  360 2018-02-12   UTTARABANK  28.0  ...  28.1  151.0       7.421   266706.0
16  423 2018-03-09   UTTARABANK  26.8  ...  27.6  454.0      15.155   563384.0

[5 rows x 12 columns]

In [320]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE  LTP*  HIGH  ...   YCP  TRADE  VALUE (mn)     VOLUME
DATE                                      ...                                    
2018-01-10  403   UTTARABANK  24.9  25.4  ...  25.0  248.0       6.841   271956.0
2018-01-11  380   UTTARABANK  29.8  30.1  ...  29.5  751.0      44.419  1522151.0
2018-02-10  402   UTTARABANK  25.4  25.4  ...  25.1  150.0       4.143   164334.0

[3 rows x 11 columns]

In [321]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [322]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [323]:
df['volatility'] = df['volatility']*100


In [324]:
import matplotlib.pyplot as plt 

In [325]:
df=df.reset_index()['volatility']

In [326]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [327]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [328]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [329]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [330]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression



In [331]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [332]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [333]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [334]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [335]:
model4.fit(X_train, y_train)

[13:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [336]:

train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [337]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [338]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [339]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [340]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [341]:
Y_test =scaler.inverse_transform(Y_test)

In [342]:
Y_test

array([[4.06905789, 4.03684079, 4.03387517, 3.3017171 , 3.3017171 ,
        3.3017171 , 3.3017171 , 3.3017171 , 3.3017171 , 3.3017171 ,
        3.3017171 , 3.3017171 , 3.3017171 , 3.96639131, 4.02715399,
        4.31542589, 4.31542589, 4.06679838, 3.39449349, 4.08903654,
        4.08903654, 4.08903654, 4.87371286, 5.21954105, 5.27680984,
        5.85979009, 6.02122648, 6.14516372, 6.37828201, 6.41804028,
        6.76304334]])

In [343]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict2.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict3.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict3.size))



In [344]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [345]:
test_predict1

array([[4.40050117, 4.29932555, 4.2928208 , 4.2763167 , 3.39671507,
        3.38641814, 3.37518702, 3.38293277, 3.38609511, 3.37811722,
        3.34541266, 3.3420952 , 3.343753  , 3.34713783, 4.34420713,
        4.35746121, 4.36126432, 4.35739806, 4.26553714, 3.32267441,
        3.88754081, 4.08720003, 4.2515611 , 4.41271767, 4.4139317 ,
        4.39186278, 4.39094739, 4.32990855, 4.1606229 , 4.05790855,
        4.05319898]])

In [346]:
test_predict2

array([[4.38625401, 4.26887381, 4.26016118, 4.2494589 , 3.39765367,
        3.37421138, 3.37283581, 3.36902195, 3.35661715, 3.35661715,
        3.35661715, 3.35974639, 3.36342469, 3.36342469, 4.36447535,
        4.41348303, 4.38625401, 4.38262398, 4.26021206, 3.4439559 ,
        3.94491766, 4.05284788, 4.2330029 , 4.36970083, 4.30440139,
        4.2827171 , 4.29260417, 4.24728172, 4.11332891, 4.01946524,
        3.99721589]])

In [347]:
test_predict3

array([[4.23309478, 4.03826298, 4.03636106, 4.02343086, 3.19846761,
        3.20622639, 3.28694077, 3.16976236, 3.10125645, 3.2005327 ,
        3.15525219, 3.23063809, 3.28105309, 3.31899485, 4.20003198,
        4.1824817 , 4.20023066, 4.20465012, 4.15555993, 3.29736132,
        3.78612838, 4.01059649, 4.10227029, 4.23859747, 4.28307888,
        4.26483324, 4.23547987, 4.27001551, 4.06460282, 3.9934932 ,
        4.0108407 ]])

In [348]:
test_predict4

array([[4.3752446, 4.1910133, 4.1792774, 4.1845837, 3.5279782, 3.515873 ,
        3.4990747, 3.3825495, 3.299438 , 3.4186363, 3.4186363, 3.4204004,
        3.3920019, 3.3834465, 4.0444613, 4.049614 , 4.051691 , 4.0620284,
        4.1429276, 3.3621366, 3.7791593, 3.9797804, 4.0500007, 4.201706 ,
        4.2541995, 4.2627554, 4.256441 , 4.2657223, 4.0887623, 4.0232086,
        4.0810905]], dtype=float32)

In [349]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

0.9987558285011238

In [350]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

1.0256719132695744

In [351]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

1.0327754357503702

In [352]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

1.0304179365817308

In [353]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 0.648297


In [354]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 0.662147


In [355]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 0.658314


In [356]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 0.682357
